## Feature Engineering : part.1

***Ce script a pour but de charger, de traiter et de sauvegarder les données brutes que nous avons à disposition.***  
Les données se décomposent de la manière suivante : 
- Une base de donnée par année enregistrant toutes les transactions des donations
- Une base de donnée des contacts de l'ONG

#### Importation des librairies

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Changement des options
pd.options.display.max_columns = None

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


#### Chargement et lecture des données
On lit les données préalablement enregistrées dans le dossier 'PATH_DATA'.  
Cette opération peut prendre un peu de temps en raison du volume des données.

In [2]:
# Chemin vers les données, par défaut '../data'
PATH_DATA = "../data"

In [3]:
# Lecture des données
def read_dons(data_filepath):
    """
    Lit la base de donnée des dons dans le répertoire 'data_filepath'
    """
    dons2014 = pd.read_excel(os.path.join(data_filepath,'Dons_2014.xlsx'))
    dons2015 = pd.read_excel(os.path.join(data_filepath,'Dons_2015.xlsx'))
    dons2016 = pd.read_excel(os.path.join(data_filepath,'Dons_2016.xlsx'))
    dons2017 = pd.read_excel(os.path.join(data_filepath,'Dons_2017.xlsx'))
    dons2018 = pd.read_excel(os.path.join(data_filepath,'Dons_2018.xlsx'))
    dons = pd.concat([dons2014, dons2015, dons2016, dons2017, dons2018])
    
    dons_columns = ['NUMERO_LOT', 'DATE_DE_RECEPTION', 'ID_CONTACT', 
                    'MONTANT_MOUVEMENT', 'CAM_CODE', 'OPERATION', 'SEGMENT',
                    'MOUVEMENT_ID', 'VENTILATION_ID', 'OFT_CODE', 'MODE_DE_PAIEMENT',
                    'MONTANT_VENTILATION', 'RF']
    dons.columns = dons_columns
    dons.set_index('VENTILATION_ID', inplace=True)
    
    return dons

def read_contacts(data_filepath):
    """
    Lit la base de données contacts dans le répertoire 'data_filepath'
    On extrait également le "SEGMENT D'ORIGINE" en vue du cleaning des données
    """
    contacts = pd.read_excel(os.path.join(data_filepath,'Tous_contacts.xlsx'), index_col=0)
    contacts.set_index('ID_CTC', inplace=True)

    contacts_columns = ['TYPE_CONTACT', 'DATE_NAISSANCE', 'DATE_CREATION', 'DECEDES',
                        'ADRESSE_POSTALE', 'PAYS', 'EMAIL_ADRESSE', 'EMAIL', 'OPTIN', 'TELEPHONE',
                        'STOP_TEL', 'STOP_MAILING', 'STOP_GENERAL', 'NPAI2', 'SEGMENT_ORIGINE',
                        'NOMBRE_DONS', 'MONTANT_DONS', 'DATE_PREMIER_DON', 'DATE_DERNIER_DON',
                        'ORIGINE_PREMIER', 'ORIGINE_DERNIER', 'MONTANT_DON_PREMIER',
                        'MONTANT_DON_DERNIER', 'NOMBRE_DONS_ANNEE_N', 'NOMBRE_DONS_ANNEE_N1',
                        'NOMBRE_DONS_ANNEE_N2', 'NOMBRE_DONS_ANNEE_N3', 'NOMBRE_DONS_ANNEE_N4',
                        'MONTANT_DONS_ANNEE_N', 'MONTANT_DONS_ANNEE_N1',
                        'MONTANT_DONS_ANNEE_N2', 'MONTANT_DONS_ANNEE_N3',
                        'MONTANT_DONS_ANNEE_N4', 'PA_ACTIF', 'DATE_DEBUT_PA_ACTIF',
                        'DATE_FIN_PA_ACTIF', 'CYCLE_PA_ACTIF', 'DATE_DERNIER_PA_ACTIF',
                        'ORIGINE_PA_ACTIF', 'NOMBRE_PRELEVEMENT', 'MONTANT_PRELEVEMENT',
                        'DATE_PREMIER_PRELEVEMENT', 'DATE_DERNIER_PRELEVEMENT', 'NOMBRE_ACHAT',
                        'MONTANT_ACHAT_MAX', 'MONTANT_ACHAT_MIN', 'DATE_PREMIER_ACHAT',
                        'DATE_DERNIER_ACHAT', 'NOMBRE_COTISATION', 'MONTANT_COTISATION',
                        'DATE_PREMIERE_COTISATION', 'DATE_DERNIERE_COTISATION',
                        'NOMBRE_PETITION', 'DATE_PREMIERE_PETITION', 'DATE_DERNIERE_PETITION']
    contacts.columns = contacts_columns
    
    segment_origine = pd.read_excel(os.path.join(data_filepath, 'Tous_contacts.xlsx'), sheet_name="Lexique codes origine", usecols='A:C')
    segment_origine.set_index('SEGMENT_ORIGINE', inplace=True)
    
    return contacts, segment_origine

def read_mails(data_filepath):
    """
    Lit la base de données mails dans le répertoire 'data_filepath'
    """
    mails = pd.read_excel(os.path.join(data_filepath, 'stat_mailjet.xlsx'))
    
    mails_columns = ['ID', 'EMAIL', 'OPEN', 'CLICK', 'SENT', 'BOUNCE', 
                     'BLOCKED', 'SPAM', 'UNSUB', 'TOTAL']
    mails.columns = mails_columns
    
    return mails
    
dons = read_dons(PATH_DATA)
contacts, segment_origine = read_contacts(PATH_DATA)
mails = read_mails(PATH_DATA)

dons.head()

,NUMERO_LOT,DATE_DE_RECEPTION,ID_CONTACT,MONTANT_MOUVEMENT,CAM_CODE,OPERATION,SEGMENT,MOUVEMENT_ID,OFT_CODE,MODE_DE_PAIEMENT,MONTANT_VENTILATION,RF
VENTILATION_ID,,,,,,,,,,,,
2415602.0,29770,2014-01-10,713843,10.0,PA,PA,PA,1116673.0,DON,PRE,10.0,Oui
2415603.0,29770,2014-01-10,818435,7.0,PA,PA,PA,1116674.0,DON,PRE,7.0,Oui
2415604.0,29770,2014-01-10,811465,10.0,PA,PA,PA,1116675.0,DON,PRE,10.0,Oui
2415605.0,29770,2014-01-10,818439,10.0,PA,PA,PA,1116676.0,DON,PRE,10.0,Oui
2415606.0,29770,2014-01-10,818443,0.0,PA,PA,PA,1116677.0,DON,PRE,0.0,Oui


#### Traitement des données
Pour chaque dataset (en particulier le dataset des contacts), on effectue des opérations de nettoyage de données ainsi que de création de variables permettant de faciliter l'exploitation de la base de données.

In [4]:
# Traitement des données
def process_dons(df_dons):
    """
    Traite le dataframe des dons
    """
    df_dons.MONTANT_MOUVEMENT.fillna(0, inplace=True)
    df_dons.MODE_DE_PAIEMENT.fillna('SIGN', inplace=True)
    df_dons.dropna(inplace=True)
    df_dons.MODE_DE_PAIEMENT.replace('CBW', 'CB', inplace=True)
    
    return df_dons

def process_contacts(df_contacts, df_segment_origine):
    """
    Traite le dataframe des contacts
    """
    # Transformation des colonnes au format 'datetime'
    df_contacts.DATE_CREATION = pd.to_datetime(df_contacts.DATE_CREATION, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_DEBUT_PA_ACTIF = pd.to_datetime(df_contacts.DATE_DEBUT_PA_ACTIF, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_DERNIER_ACHAT = pd.to_datetime(df_contacts.DATE_DERNIER_ACHAT, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_DERNIER_DON = pd.to_datetime(df_contacts.DATE_DERNIER_DON, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_DERNIER_PA_ACTIF = pd.to_datetime(df_contacts.DATE_DERNIER_PA_ACTIF, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_DERNIER_PRELEVEMENT = pd.to_datetime(df_contacts.DATE_DERNIER_PRELEVEMENT, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_DERNIERE_COTISATION = pd.to_datetime(df_contacts.DATE_DERNIERE_COTISATION, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_DERNIERE_PETITION = pd.to_datetime(df_contacts.DATE_DERNIERE_PETITION, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_FIN_PA_ACTIF = pd.to_datetime(df_contacts.DATE_FIN_PA_ACTIF, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_NAISSANCE = pd.to_datetime(df_contacts.DATE_NAISSANCE, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_PREMIER_ACHAT = pd.to_datetime(df_contacts.DATE_PREMIER_ACHAT, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_PREMIER_DON = pd.to_datetime(df_contacts.DATE_PREMIER_DON, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_PREMIER_PRELEVEMENT = pd.to_datetime(df_contacts.DATE_PREMIER_PRELEVEMENT, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_PREMIERE_COTISATION = pd.to_datetime(df_contacts.DATE_PREMIERE_COTISATION, format='%Y%m%d', errors='coerce')
    df_contacts.DATE_PREMIERE_PETITION = pd.to_datetime(df_contacts.DATE_PREMIERE_PETITION, format='%Y%m%d', errors='coerce')
    
    # Traitement des données textuelles 
    df_contacts.EMAIL = df_contacts.EMAIL.str.lower().map(dict(oui=1, non=0))
    df_contacts.DECEDES = df_contacts.DECEDES.str.lower().map(dict(oui=1, non=0)).fillna(0)
    df_contacts.OPTIN = df_contacts.OPTIN.str.lower().map(dict(oui=1, non=0))
    df_contacts.TELEPHONE = df_contacts.TELEPHONE.str.lower().map(dict(oui=1, non=0)).fillna(0)
    df_contacts.ADRESSE_POSTALE = df_contacts.ADRESSE_POSTALE.str.lower().map(dict(oui=1, non=0))
    df_contacts.STOP_TEL = df_contacts.STOP_TEL.str.lower().map(dict(oui=1, non=0)).fillna(0)
    df_contacts.STOP_MAILING = df_contacts.STOP_MAILING.str.lower().map(dict(oui=1, non=0)).fillna(0)
    df_contacts.STOP_GENERAL = df_contacts.STOP_GENERAL.str.lower().map(dict(oui=1, non=0)).fillna(0)
    df_contacts.NPAI2 = df_contacts.NPAI2.str.lower().map(dict(oui=1, non=0)).fillna(0)
    
    # Traitement des dates de naissance
    year = df_contacts.DATE_NAISSANCE.apply(lambda x: x.year)
    mask1 = (year < 1900) | (year > 2005)
    mask2 = (df_contacts.DATE_NAISSANCE == datetime(1970, 1, 1))
    mask3 = (df_contacts.DATE_NAISSANCE == datetime(1900, 1, 1))
    mask = (mask1 | mask2 | mask3)
    df_contacts.loc[mask, 'DATE_NAISSANCE'] = np.nan
    
    # Remplissage des données manquantes
    df_contacts.MONTANT_DONS.fillna(0, inplace=True)
    df_contacts.MONTANT_DON_PREMIER.fillna(0, inplace=True)
    df_contacts.MONTANT_DON_DERNIER.fillna(0, inplace=True)
    df_contacts.MONTANT_DONS.fillna(0, inplace=True)
    
    # Création de nouvelles features
    df_contacts['MORAL'] = df_contacts.TYPE_CONTACT.map(dict(MORAL=1, PHYSIQUE=0))
    df_contacts['PHYSIQUE'] = df_contacts.TYPE_CONTACT.map(dict(MORAL=0, PHYSIQUE=1))
    df_contacts['NEW_ADRESSE_POSTALE'] = df_contacts[['ADRESSE_POSTALE', 'NPAI2']].apply(lambda x: x[0] if x[1] == 0 else 0, axis=1)
    df_contacts['CANAL_ACQUISITION'] = df_contacts.SEGMENT_ORIGINE.map(df_segment_origine.to_dict()["Canal d'acquisition"])
    df_contacts['CANAL_ACQUISITION_AGG'] = df_contacts.SEGMENT_ORIGINE.map(df_segment_origine.to_dict()["Canal d'acquisition agrégé"])
    df_contacts['CANAL_ACQUISITION_PREMIER'] = df_contacts.ORIGINE_PREMIER.map(df_segment_origine.to_dict()["Canal d'acquisition"])
    df_contacts['CANAL_ACQUISITION_AGG_PREMIER'] = df_contacts.ORIGINE_PREMIER.map(df_segment_origine.to_dict()["Canal d'acquisition agrégé"])
    df_contacts['CANAL_ACQUISITION_DERNIER'] = df_contacts.ORIGINE_DERNIER.map(df_segment_origine.to_dict()["Canal d'acquisition"])
    df_contacts['CANAL_ACQUISITION_AGG_DERNIER'] = df_contacts.ORIGINE_DERNIER.map(df_segment_origine.to_dict()["Canal d'acquisition agrégé"])
    
    df_contacts['AGE'] = df_contacts.DATE_NAISSANCE.apply(lambda x: np.floor((datetime.today()-x).days/365))

    def tranche_age(age):
        tr = None
        if age <= 25:
            tr = '0-25'
        elif 25 < age <= 45:
            tr = '25-45'
        elif 45 < age <= 65:
            tr = '45-65'
        elif age > 65:
            tr = '65+'
        return tr

    df_contacts['TRANCHE_AGE'] = df_contacts.AGE.apply(tranche_age)
    
    return df_contacts

dons = process_dons(dons)
contacts = process_contacts(contacts, segment_origine)

#### Extraction du segment 'donateurs réguliers'
On rappelle que les donateurs de l'ONG sont organisés selon 4 catégories dans l'ordre d'importance décroissant ci-dessous : 
- les donateurs réguliers : effectuants des donations régulières via des prélèvements automatiques
- les donateurs occasionnels : effectuants des donations de temps à autre
- les donateurs évènementiels : participants uniquement aux évènements de l'ONG (Trail Walker & Winter Trail)
- les signataires : étant signés une pétition (présent dans la base de données mais n'ayant jamais donné)

On extrait ici les donateurs réguliers : qui sont les donateurs sur lesquels nous allons faire nos analyses.

In [5]:
def labelisation_donators(df_contacts, df_dons):
    """
    Labellise les donateurs selon les 4 catégories ('réguliers', 'occasionnels', 'évènementiels', 'signataires')
    """
    cont_oft = pd.crosstab(df_dons.ID_CONTACT, df_dons.OFT_CODE)
    cont_cam = pd.crosstab(df_dons.ID_CONTACT, df_dons.CAM_CODE)

    # Définition de chaque segment
    signataires = cont_oft[(cont_oft.ACH == 0) & (cont_oft.COTI == 0) & (cont_oft.DON == 0)].index
    reguliers = cont_cam[cont_cam.PA != 0].index
    occa = cont_cam[(cont_cam.PA == 0) & (cont_cam.TW == 0) & (cont_cam.WTR == 0) & (cont_cam.EVENT == 0)].index
    event = cont_cam[((cont_cam.TW != 0) | (cont_cam.WTR != 0) | (cont_cam.EVENT != 0)) &
                     (cont_cam.ADH == 0) & (cont_cam.CIEL == 0) & (cont_cam.DIV == 0) &
                     (cont_cam.F == 0) & (cont_cam.FE == 0) & (cont_cam.FMD == 0) &
                     (cont_cam.FTEL == 0) & (cont_cam.HF == 0) & (cont_cam.IA == 0) &
                     (cont_cam.O == 0) & (cont_cam.OI == 0) & (cont_cam.PA == 0) &
                     (cont_cam.PART == 0) & (cont_cam.PLAQ == 0) & (cont_cam.PROGJE == 0) &
                     (cont_cam.PROGSE == 0) & (cont_cam.PWEB == 0) & (cont_cam.RA == 0) &
                     (cont_cam.RF == 0) & (cont_cam.SITE == 0) & (cont_cam.TEST_PET == 0) &
                     (cont_cam.TTEL == 0) & (cont_cam.URGENCE == 0)].index

    # Labelisation
    df_contacts['REGULIER_EVENT_OCCA'] = np.nan
    df_contacts.loc[df_contacts.index.isin(signataires), 'REGULIER_EVENT_OCCA'] = 'Signataires'
    df_contacts.loc[df_contacts.index.isin(reguliers), 'REGULIER_EVENT_OCCA'] = 'Régulier'
    df_contacts.loc[df_contacts.index.isin([cont for cont in occa if cont not in signataires]) , 'REGULIER_EVENT_OCCA'] = 'Occasionnel'
    df_contacts.loc[df_contacts.index.isin([cont for cont in event if cont not in signataires]), 'REGULIER_EVENT_OCCA'] = 'Evenementiel'
    
    return df_contacts

contacts = labelisation_donators(contacts, dons)

In [6]:
# Jointure entre la base de donnée 'contact' et 'dons'
data = dons.join(contacts, on='ID_CONTACT', how='left')

In [7]:
def extract_regular_donators(df_data):
    """
    Extrait les donateurs réguliers de la base de données précédentes
    On ne garde que les colonnes utiles pour nos modèles
    On peut finir de nettoyer ces données avec des règles propres à ce segment
    """
    # Extraction des données
    cols = ['DATE_DE_RECEPTION', 'ID_CONTACT', 
            'MONTANT_MOUVEMENT', 'CAM_CODE', 'OPERATION', 'SEGMENT', 'MOUVEMENT_ID', 
            'OFT_CODE', 'MODE_DE_PAIEMENT', 'MONTANT_VENTILATION']
    donators_regular = df_data.loc[df_data.REGULIER_EVENT_OCCA == 'Régulier'][cols]
    
    # Traitement des données
    donators_regular.DATE_DE_RECEPTION = pd.to_datetime(donators_regular.DATE_DE_RECEPTION, format='%Y-%m-%d', errors='coerce')
    donators_regular.MONTANT_MOUVEMENT = donators_regular.MONTANT_MOUVEMENT.fillna(0)
    donators_regular.MODE_DE_PAIEMENT = donators_regular.MODE_DE_PAIEMENT.fillna('SIGN')
    donators_regular = donators_regular.dropna()
    
    return donators_regular

dons_reg = extract_regular_donators(data)

In [20]:
def process_mails(mails, contacts):
    """
    Joint la base de données contacts et mails pour associer à chaque client, son comportement
    face aux mails envoyés
    """
    
    contact_merge = contacts.loc[:, ['EMAIL_ADRESSE']]
    contact_merge['ID_CONTACT'] = contact_merge.index
    # Jointure des dataset
    new_mail = mails.merge(contact_merge, right_on='EMAIL_ADRESSE', left_on='EMAIL', how='left')
    new_mail['RATIO_CLICK'] = np.where(new_mail.SENT != 0, new_mail.CLICK / new_mail.SENT, 0) 
    new_mail['RATIO_OPEN'] = np.where(new_mail.SENT != 0, new_mail.OPEN / new_mail.SENT, 0)
    
    new_mail = new_mail[['ID_CONTACT', 'RATIO_CLICK', 'RATIO_OPEN']]
    new_mail = new_mail.dropna()
    
    return new_mail

mails = process_mails(mails, contacts)

#### Sauvegarde des données
On enregistre les données dans le même répertoire, pour pouvoir les exploiter avec les autres scripts.

In [34]:
def save_cleaned_dataframe(data_filepath, df_data, df_dons_reg, df_mails):
    """
    On sauvegarde les données traitées dans le répertoire 'data_filepath':
    - les données contacts et dons, préalablement fusionnées
    - les données des donateurs réguliers, utiles à nos modèles
    """
    df_data.to_csv(os.path.join(data_filepath, 'dons_contact_cleaned.csv'), index=False)
    df_dons_reg.to_csv(os.path.join(data_filepath, 'dons_reguliers_cleaned.csv'), index=True)
    df_mails.to_csv(os.path.join(data_filepath, 'mails_cleaned.csv'), index=False)
    
    print("data saved !")

save_cleaned_dataframe(PATH_DATA, data, dons_reg, mails)

data saved !
